# **ПРОГНОЗ СТОИМОСТИ АВТОМОБИЛЕЙ**
# Парсинг
## Задача
Необходимо создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам. Для оценки использовать метрику MAPE

С помощью данного ноутбука мы провели сбор данных с помощью парсинга для обучения.

Также в этом проекте мы использовали:

* Ноутбук, через который парсили https://www.kaggle.com/tatianamukhidaeva/prj5-kirill-n-tanya-m-parser
* Спарсенный датасет https://www.kaggle.com/tatianamukhidaeva/cars-df-v2
* Ноутбук, в котором провели EDA https://www.kaggle.com/tatianamukhidaeva/prj5-kirill-n-tanya-m-eda
* Ноутбук, в котором провели обучение https://www.kaggle.com/tatianamukhidaeva/prj5-kirill-n-tanya-m

## Библиотеки

In [1]:
pip install BeautifulSoup4

     |████████████████████████████████| 115 kB 5.0 MB/s 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import time
import datetime
import requests as r
import json
from bs4 import BeautifulSoup
import warnings
warnings.simplefilter('ignore')


In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [4]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

## Анализ тестового  датасета

По легенде тестовый датасет состоит из автомобилей из истории продаж, и он слишком мал для тренировочных данных. Но все же посмотреть мы его можем, чтобы лучше понять, какие данные необходимо собрать.

In [5]:
VERSION    = 16
# DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

In [6]:
test = pd.read_csv(DIR_TEST+'test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [7]:
test.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


В парсинге могут быть любые модели, в тесте представлены самые популярные. Про город достоверно неизвестно. Только из вводного вебинара предполагаем взять только Москву,  так как по другим регионам цены не сильно меняются.

In [8]:
test_brands = test['brand'].unique()
test_brands = [s.lower() for s in test_brands]
test_brands

['skoda',
 'audi',
 'honda',
 'volvo',
 'bmw',
 'nissan',
 'infiniti',
 'mercedes',
 'toyota',
 'lexus',
 'volkswagen',
 'mitsubishi']

##  Сам парсинг
Зафиксируем адреса страниц с поиском объявлений о продаже авто по Москве, с пробегом и по маркам, которые есть в тестовых данных.

In [9]:
URLS = [
    'https://auto.ru/moskva/cars/skoda/used/',
    'https://auto.ru/moskva/cars/audi/used/',
    'https://auto.ru/moskva/cars/honda/used/',
    'https://auto.ru/moskva/cars/volvo/used/',
    'https://auto.ru/moskva/cars/bmw/used/',
    'https://auto.ru/moskva/cars/nissan/used/',
    'https://auto.ru/moskva/cars/infiniti/used/',
    'https://auto.ru/moskva/cars/mercedes/used/',
    'https://auto.ru/moskva/cars/toyota/used/',
    'https://auto.ru/moskva/cars/lexus/used/',
    'https://auto.ru/moskva/cars/volkswagen/used/',
    'https://auto.ru/moskva/cars/mitsubishi/used/'
]
pages_url_list=[]


Получим список ссылок pages_url_list. Чтобы сайт нас не блокировал, добавим sleep на 10s.
Чтобы получить побольше объявлений увеличимрадиус поиска до 1000 км. Спарсим как можно больше ссылок, так как в тесте 34686 машин. Для лучшего обучения хотелось бы в тренировочных данных тоже иметь достаточное количество объявлений.

In [10]:
for START_URL in URLS:
    print('сейчас парсим по ссылке:', START_URL)
    # Чтобы получить побольше объявлений увеличимрадиус поиска до 1000 км.
    response = r.get( START_URL, cookies={'gradius': '1000'}) 
    response.encoding = 'utf-8'
    page = BeautifulSoup(response.text, 'html.parser')
    pagins = page.find('span',{'class':'ControlGroup ControlGroup_responsive_no ControlGroup_size_s ListingPagination-module__pages'}).findAll('a')
    pages_count = int(pagins[-1].text)
    
    # пройдем по всем страницам
    for p in range(1, pages_count+1): 
        time.sleep(10) # задержка перед запросом
        response = r.get( START_URL+'?page=%s'%p, cookies={'gradius': '1000'})
        response.encoding = 'utf-8'
        page = BeautifulSoup(response.text, 'html.parser')
        for link in page.findAll('a',{"class": "Link ListingItemTitle__link"}):
            try:
                if link['href'] not in pages_url_list: 
                    pages_url_list.append(link['href'])
            except:
                pass
        print('Добавлено в список:',len(pages_url_list), 'записей')   
        
with open('pages_url_list2.txt', 'w') as f:
    for item in pages_url_list:
        f.write("%s\n" % item)

сейчас парсим по ссылке: https://auto.ru/moskva/cars/skoda/used/
Добавлено в список: 38 записей
Добавлено в список: 74 записей
Добавлено в список: 111 записей
Добавлено в список: 144 записей
Добавлено в список: 159 записей
Добавлено в список: 197 записей
Добавлено в список: 234 записей
Добавлено в список: 272 записей
Добавлено в список: 310 записей
Добавлено в список: 348 записей
Добавлено в список: 386 записей
Добавлено в список: 423 записей
Добавлено в список: 461 записей
Добавлено в список: 499 записей
Добавлено в список: 537 записей
Добавлено в список: 574 записей
Добавлено в список: 611 записей
Добавлено в список: 648 записей
Добавлено в список: 685 записей
Добавлено в список: 722 записей
Добавлено в список: 759 записей
Добавлено в список: 796 записей
Добавлено в список: 833 записей
Добавлено в список: 870 записей
Добавлено в список: 907 записей
Добавлено в список: 944 записей
Добавлено в список: 981 записей
Добавлено в список: 1018 записей
Добавлено в список: 1055 записей
Добавле

In [11]:
# pages_url_list_before_mers = pages_url_list
pages_url_list_before_other = pages_url_list
print(len(pages_url_list_before_mers))
print(len(pages_url_list_before_other))
# pages_url_list_before_all = pages_url_list_before_mers + pages_url_list_before_other
print(len(pages_url_list_before_all))

NameError: name 'pages_url_list_before_mers' is not defined

In [ ]:
# запомним список ссылок, чтобы не парсить его больше
# with open('pages_url_list1.txt', 'w') as f:
#     for item in pages_url_list:
#         f.write("%s\n" % item)

Заметили, что на странице хранятся json файлы, из которых сайт получает данные. Можем воспользоваться ими. Используем первый ld+json, и id=initial-state, чтобы получить комплектацию.

In [ ]:
pages_url_list = pages_url_list_before_all
cars_list = []
page = ''
for item in pages_url_list:
    response = r.get(item)
    response.encoding = 'utf8'
    page = BeautifulSoup(response.text, 'html.parser')
    if page.find('div', class_='CardSold') == None:
        try:
            json_data = json.loads(
                page.find('script', type="application/ld+json").string)
        except:
            print(item)
            pass
        cls_str = 'CardInfoRow_'
        span_str = 'CardInfoRow__cell'
        try:
            cars_item = {
            'bodyType': json_data['bodyType'],
            'brand': json_data['brand'],
            'car_url': json_data['offers']['url'],
            'color': json_data['color'],
            'description': json_data['description'],
            'engineDisplacement': json_data['vehicleEngine']['engineDisplacement'],
            'enginePower': json_data['vehicleEngine']['enginePower'],
            'fuelType': json_data['fuelType'],
            'image': json_data['image'],
            'mileage': page.find(
                'li', class_=cls_str+'kmAge').find_all('span')[1].text,
            'modelDate': json_data['modelDate'],
            'model_name': json_data['name'],
            'name': json_data['vehicleEngine']['name'],
            'numberOfDoors': json_data['numberOfDoors'],
            'parsing_unixtime': int(time.time()),
            'priceCurrency': json_data['offers']['priceCurrency'],
            'productionDate': json_data['productionDate'],
            'sell_id': page.find(
                'div', title='Идентификатор объявления').text,
            'vehicleConfiguration': json_data['vehicleConfiguration'],
            'vehicleTransmission': json_data['vehicleTransmission'],
            'Владельцы': page.find(
                'li', class_=cls_str+'ownersCount').find_all('span')[1].text,
            'ПТС': page.find(
                'li', class_=cls_str+'pts').find_all('span')[1].text,
            'Привод': page.find(
                'li', class_=cls_str+'drive').find_all('span')[1].text,
            'Руль': page.find(
                'li', class_=cls_str+'wheel').find_all('span')[1].text,
            'Состояние': page.find(
                'li', class_=cls_str+'state').find_all('span')[1].text,
            'Таможня': page.find(
                'li', class_=cls_str+'customs').find_all('span')[1].text,
            'price': page.find(
                'span', class_='OfferPriceCaption__price').text
            }
            cars_item['equipmentGroups'] = None
            
            try:
                json_data2 = json.loads(page.find('script', id="initial-state").contents[0])
                cars_item['equipmentGroups'] = json_data2['card']['vehicle_info']['equipmentGroups']
            except:
                print('Нет описания комплектации:', item)
            cars_list.append(cars_item)
#             print(json_data['color'])
#             print(json_data2['card']['vehicle_info']['equipmentGroups'])
        except:
            print(item)
            pass

   # else cars_list = []

    #print('Ожидаю 0.1 секунды...')
    time.sleep(2)

Парсинг выполнялся более 36 часов, но зато не прерывался

In [ ]:
len(cars_list)

Переведем полученные данные в датафрейм и сохраним в файл

In [ ]:
# переводим в удобный для нас вид
df = pd.DataFrame(cars_list)
# проверяем на дубликаты, можно по car_url
df.drop_duplicates('car_url', inplace = True)
# сохраняем в csv
df.to_csv('cars_df.csv', index=False)
df

In [ ]:
# df[df.duplicated('car_url')].sort_values('car_url')
df.drop_duplicates('car_url', inplace = True)
df = pd.read_csv('cars_df.csv')
df

In [ ]:
display(df.iloc[0])
display(df.iloc[0]['equipmentGroups'])
df.iloc[0]['description']

# Вывод
1. Много данных парсится очень долго, пришлось запускать параллельно. Даже обход и сбор ссылок на объявления затратен по времени
2. Радиус поиска пришлось увеличить, для этого установили значение куков.
3. Для получения нужных данных использовали json файлы сайта.
4. Цены 2021 года не совсем корректные, поэтому в EDA подкорректируем их с учетом инфляции с осени 2020 года.